# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [43]:
from pyspark.sql import  SparkSession

In [44]:
spark = SparkSession.builder.appName('hyundai').getOrCreate()

In [45]:
data = spark.read.csv('cruise_ship_info.csv',header=True,inferSchema=True)

In [46]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [47]:
data.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|               158|               158|              158|              158|               158|              158|              158|
|   mean|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|                 4|             2.329|             0.66|             2.79|              0.33|             17.7|             0.59|
|    max|                48

In [48]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [49]:
from pyspark.ml.feature import VectorAssembler,StringIndexer, OneHotEncoder

In [50]:
indexer = StringIndexer(inputCol="Cruise_line",outputCol="clIndex")

In [51]:
encoder = OneHotEncoder(inputCol='clIndex',outputCol='clVec')

In [52]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [53]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density','clVec'],outputCol='features')

In [54]:
from pyspark.ml.regression import LinearRegression

In [55]:
from pyspark.ml import Pipeline

In [70]:
lr = LinearRegression(labelCol='crew')

In [57]:
pipeline = Pipeline(stages=[indexer,encoder,assembler])

In [ ]:
pipe = pipeline.fit(data)

In [58]:
clean = pipe.transform(data)

In [73]:
col_data = clean.select('features','crew')

In [74]:
final_data = col_data.na.drop()

DataFrame[features: vector, crew: double]

In [75]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [76]:
model = lr.fit(train_data)

In [77]:
results = model.evaluate(test_data)

In [78]:
results.rootMeanSquaredError

0.5462683684719782

In [79]:
results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  1.0078275112294506|
|  0.1649939002645251|
|  0.2230988231002229|
|  0.3291500482190681|
|-0.17469146742676855|
|  0.5148452967114725|
|-0.22898303682740462|
|-0.33964884638585247|
|-0.41089041500723056|
| -0.4500152676858633|
| -0.9977062428907413|
|-0.13878963151518953|
|0.002575357723312...|
|-0.04033889679947...|
|-0.06894839981466028|
|  -0.163881803700745|
|  0.3751145635213291|
|  0.4379107142981362|
| -0.7678719656089585|
| -0.4575315784227705|
+--------------------+
only showing top 20 rows



In [80]:
results.r2

0.980789792903909